In [1]:
import os
import time
import shutil

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from hybrid_net import DataTransformer, DataLoaderMaker, ModelArchitecture, MetricMeters, HyperParameterSchedulers, Losses

from PIL import Image

# Important Parameters

In [2]:
images = "./data-local/images/cifar10/by-image/"
labels = "./data-local/labels/cifar10/1000_balanced_labels/00.txt"
train_subdir = "train"
eval_subdir = "val"
checkpoint_path = "./checkpoints/"
device = "cuda" if torch.cuda.is_available() else 'cpu'
workers = 4
NO_LABEL = -1
global_step = 0
ema_decay = 0.999
print_freq = 2
best_prec1 = 0
start_epoch = 0
arch = "hybridNet1"


# specified in the paper
initial_lr = 0.003
initial_beta1 = 0.9

# batch_size is 100 in paper
batch_size = 10

# batch_size is 20 in paper
labeled_batch_size = 1

# total epochs not specified for 1000 label CIFAR training in paper
total_epochs = 3

# Helper Functions

In [3]:
# function to update the secondary model (used for stability loss)

def update_ema_variables(model, ema_model, alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)

In [4]:
# function to save checkpoints of the model after every epoch

def save_checkpoint(state, is_best, dirpath, epoch):
    filename = 'checkpoint-{}.ckpt'.format(epoch)
    checkpoint_path = os.path.join(dirpath, filename)
    best_path = os.path.join(dirpath, 'best.ckpt')
    torch.save(state, checkpoint_path)
    print('--- checkpoint saved to %s ---'.format(checkpoint_path))
    if is_best:
        shutil.copyfile(checkpoint_path, best_path)
        print('--- checkpoint copied to %s ---'.format(best_path))

# DataLoader Preparation

In [5]:
train_transformation, eval_transformation = DataTransformer.transformer()

In [6]:
train_loader, eval_loader = DataLoaderMaker.create_data_loaders(train_transformation, 
                                                eval_transformation, 
                                                images, 
                                                train_subdir, 
                                                eval_subdir,
                                                labels,
                                                batch_size,
                                                labeled_batch_size,
                                                workers)

# Model Preparation

In [7]:
# 'model' is the actual model to be trained
# 'ema_model' is a duplicate model for stability loss as mentioned in paper

model = ModelArchitecture.create_model()
ema_model = ModelArchitecture.create_model(ema=True)

# Train and Validation Functions

In [8]:
def train(train_loader, model, ema_model, optimizer, epoch, ema_decay, total_epochs, print_freq = 2):
    global global_step
    
    
    # initialize loss functions
    class_criterion = nn.CrossEntropyLoss(size_average=False, ignore_index=NO_LABEL).to(device)
    reconstruction_criterion = Losses.symmetric_mse_loss   
    stability_criterion = Losses.softmax_mse_loss
    
    # initialize evaluation metric meter
    meters = MetricMeters.AverageMeterSet()

    # switch to train mode
    model.train()
    ema_model.train()

    end = time.time()
    
    for i, ((input, ema_input), target) in enumerate(train_loader):
        # measure data loading time
        meters.update('data_time', time.time() - end)
        
        # update hyperparameters according to details given in the paper
        HyperParameterSchedulers.adjust_learning_rate(optimizer, initial_lr, epoch, total_epochs)
        HyperParameterSchedulers.adjust_beta_1(optimizer, initial_beta1, epoch, total_epochs)
        if epoch == 0:
            lambda_c = HyperParameterSchedulers.exponential_increase(i, 800)
        else: 
            lambda_c = 1.0
        lambda_r = 100 * HyperParameterSchedulers.adjust_lambda_r(epoch, 0.25 * total_epochs, 0.8 * total_epochs, total_epochs)
        lambda_s = HyperParameterSchedulers.exponential_decrease(epoch, 0.95 * total_epochs, total_epochs)
        meters.update('lr', optimizer.param_groups[0]['lr'])
        meters.update('beta1', optimizer.param_groups[0]['betas'][0])
        meters.update('lambda_c', lambda_c)
        meters.update('lambda_r', lambda_r)
        meters.update('lambda_s', lambda_s)
        
        # prepare input and target 
        input_var = torch.autograd.Variable(input)
        ema_input_var = torch.autograd.Variable(ema_input, volatile=True)
        target_var = torch.autograd.Variable(target.cuda(async=True))
        
        input_var = input_var.to(device)
        ema_input_var = ema_input_var.to(device)

        minibatch_size = len(target_var)
        labeled_minibatch_size = target_var.data.ne(NO_LABEL).sum().float()
        
        assert labeled_minibatch_size > 0
        meters.update('labeled_minibatch_size', labeled_minibatch_size)
        
        
        # forward pass through two models
        ema_y, ema_x_c, ema_x_u = ema_model(ema_input_var)
        model_y, model_x_c, model_x_u = model(input_var)

        # get logits of outputs from the model
        ema_logit = Variable(ema_y.detach().data, requires_grad=False)
        cons_logit = model_y
        class_logit = model_y
        
        # classification loss
        class_loss = class_criterion(class_logit, target_var) / labeled_minibatch_size
        meters.update('class_loss', class_loss.data[0])
        
        ema_class_loss = class_criterion(ema_logit, target_var) / labeled_minibatch_size
        meters.update('ema_class_loss', ema_class_loss.data[0])

        # reconstruction loss
        reconstruction_loss = reconstruction_criterion(model_x_c + model_x_u, input_var)/minibatch_size
        meters.update('reconstruction_loss', reconstruction_loss)
        
        # TODO add reconstruction loss of in-between layers
        #
        #
        #
        reconstruction_loss_bl = 0
        
        # balanced reconstruction loss
        if torch.sum((model_x_c - input_var)**2) <= torch.sum((model_x_u - input_var)**2):
            reconstruction_loss = reconstruction_criterion(model_x_u + model_x_c.detach(), input_var)/minibatch_size
        else:
            reconstruction_loss = reconstruction_criterion(model_x_u.detach() + model_x_c, input_var)/minibatch_size
      
        
        # stability loss
        stability_loss = stability_criterion(cons_logit, ema_logit) / minibatch_size
        meters.update('stability_loss', stability_loss.data[0])


        # final loss as given in paper
        loss = lambda_c * class_loss + lambda_r * reconstruction_loss + lambda_r * reconstruction_loss_bl + lambda_s * stability_loss 
        
        # ensure the loss is not exploding
        assert not (np.isnan(loss.data[0]) or loss.data[0] > 1e5), 'Loss explosion: {}'.format(loss.data[0])
        
        meters.update('loss', loss.data[0])

        # evaluate performance of model after training
        prec1, prec5 = MetricMeters.accuracy(class_logit.data, target_var.data, topk=(1, 5))
        meters.update('top1', prec1[0], labeled_minibatch_size)
        meters.update('error1', 100. - prec1[0], labeled_minibatch_size)
        meters.update('top5', prec5[0], labeled_minibatch_size)
        meters.update('error5', 100. - prec5[0], labeled_minibatch_size)
        
        # evaluate performance of ema_model after training
        ema_prec1, ema_prec5 = MetricMeters.accuracy(ema_logit.data, target_var.data, topk=(1, 5))
        meters.update('ema_top1', ema_prec1[0], labeled_minibatch_size)
        meters.update('ema_error1', 100. - ema_prec1[0], labeled_minibatch_size)
        meters.update('ema_top5', ema_prec5[0], labeled_minibatch_size)
        meters.update('ema_error5', 100. - ema_prec5[0], labeled_minibatch_size)

        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        global_step += 1
        
        # update parameters of 'ema_model' as it is not trained
        update_ema_variables(model, ema_model, ema_decay, global_step)

        # measure elapsed time
        meters.update('batch_time', time.time() - end)
        end = time.time()

        # print training evaluation metrics
        if i % print_freq == 0:
            print(
                'Epoch: [{0}][{1}/{2}]\t'
                'Class {meters[class_loss]:.4f}\t'
                'Prec@1 {meters[top1]:.3f}\t'
                'Prec@5 {meters[top5]:.3f}'.format(
                    epoch, i, len(train_loader), meters=meters))

In [9]:
def validate(eval_loader, model, global_step, epoch, print_freq = 2):
    
    # initialize loss functions
    class_criterion = nn.CrossEntropyLoss(size_average=False, ignore_index=NO_LABEL).to(device)
    
    # initialize evaluation metric meter
    meters = MetricMeters.AverageMeterSet()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(eval_loader):
        # measure data loading time
        meters.update('data_time', time.time() - end)
        
        
        # prepare input and target
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target.cuda(async=True), volatile=True)

        input_var = input_var.to(device)

        minibatch_size = len(target_var)
        labeled_minibatch_size = target_var.data.ne(NO_LABEL).sum().float()
        assert labeled_minibatch_size > 0
        meters.update('labeled_minibatch_size', labeled_minibatch_size)

        # forward pass through the model
        model_y, model_x_c, model_x_u = model(input_var)
        softmax_model_y = F.softmax(model_y, dim=1)
        class_loss = class_criterion(model_y, target_var) / minibatch_size

        # measure accuracy and record loss
        prec1, prec5 = MetricMeters.accuracy(model_y.data, target_var.data, topk=(1, 5))
        meters.update('class_loss', class_loss.data[0], labeled_minibatch_size)
        meters.update('top1', prec1[0], labeled_minibatch_size)
        meters.update('error1', 100.0 - prec1[0], labeled_minibatch_size)
        meters.update('top5', prec5[0], labeled_minibatch_size)
        meters.update('error5', 100.0 - prec5[0], labeled_minibatch_size)

        # measure elapsed time
        meters.update('batch_time', time.time() - end)
        end = time.time()

        # print evaluation metric for particular batches
        if i % print_freq == 0:
            print(
                'Test: [{0}/{1}]\t'
                'Class {meters[class_loss]:.4f}\t'
                'Prec@1 {meters[top1]:.3f}\t'
                'Prec@5 {meters[top5]:.3f}'.format(
                    i, len(eval_loader), meters=meters))

    # print overall evaluation metric for the model
    print(' * Prec@1 {top1.avg:.3f}\tPrec@5 {top5.avg:.3f}'
          .format(top1=meters['top1'], top5=meters['top5']))

    return meters['top1'].avg

# Running the Model

In [10]:
# use Adam optimizer as mentioned in the paper
optimizer = torch.optim.Adam(model.parameters(), lr = initial_lr)

In [11]:
for epoch in range(start_epoch, total_epochs):
    # training
    start_time = time.time()
    train(train_loader, model, ema_model, optimizer, epoch, ema_decay, total_epochs, print_freq)
    time_elapsed = time.time() - start_time
    print('epoch training complete in {: .0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    #validation
    start_time = time.time()
    print("Evaluating the primary model:")
    prec1 = validate(eval_loader, model, global_step, epoch, print_freq = 2)
    print("Evaluating the EMA model:")
    ema_prec1 = validate(eval_loader, model, global_step, epoch + 1, print_freq = 2)
    time_elapsed = time.time() - start_time
    print('epoch validation complete in {: .0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    # best precision updation
    is_best = ema_prec1 > best_prec1
    best_prec1 = max(ema_prec1, best_prec1)
    
    # saving a checkpoint of the model after the epoch
    save_checkpoint({
        'epoch': epoch + 1,
        'global_step': global_step,
        'arch': arch,
        'state_dict': model.state_dict(),
        'ema_state_dict': ema_model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, is_best, checkpoint_path, epoch + 1)

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:64: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:67: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim

Epoch: [0][0/4888]	Class 2.0970 (2.0970)	Prec@1 100.000 (100.000)	Prec@5 100.000 (100.000)
Epoch: [0][2/4888]	Class 2.6016 (2.3435)	Prec@1 0.000 (33.333)	Prec@5 0.000 (66.667)
Epoch: [0][4/4888]	Class 2.4821 (2.3053)	Prec@1 0.000 (20.000)	Prec@5 0.000 (60.000)
Epoch: [0][6/4888]	Class 2.5454 (2.3366)	Prec@1 0.000 (14.286)	Prec@5 0.000 (57.143)
Epoch: [0][8/4888]	Class 1.8276 (2.3335)	Prec@1 100.000 (22.222)	Prec@5 100.000 (55.556)
Epoch: [0][10/4888]	Class 2.8058 (2.3715)	Prec@1 0.000 (18.182)	Prec@5 0.000 (54.545)
Epoch: [0][12/4888]	Class 1.6589 (2.3350)	Prec@1 100.000 (23.077)	Prec@5 100.000 (53.846)
Epoch: [0][14/4888]	Class 2.7905 (2.3613)	Prec@1 0.000 (20.000)	Prec@5 0.000 (53.333)
Epoch: [0][16/4888]	Class 2.1662 (2.3634)	Prec@1 0.000 (17.647)	Prec@5 100.000 (52.941)
Epoch: [0][18/4888]	Class 2.5723 (2.3804)	Prec@1 0.000 (15.789)	Prec@5 0.000 (47.368)
Epoch: [0][20/4888]	Class 1.7843 (2.3434)	Prec@1 100.000 (19.048)	Prec@5 100.000 (52.381)


RuntimeError: CUDA error: out of memory